In [ ]:
import os
import re
from torch.utils.data import Dataset
import pandas as pd
from PIL import Image
from datasets import Dataset, DatasetDict
import pandas as pd
import os
import requests
import matplotlib.pyplot as plt
import random
import torch
import numpy as np
import torchvision.transforms as transforms
from google.colab import drive
drive.mount('/content/drive')


def set_seed(seed_value=42):
    """Set seed for reproducibility for PyTorch and NumPy.

    Args:
        seed_value (int): The seed value to set for random number generators.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


set_seed(42)
print("hello")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
hello


In [ ]:

path_project = '/content/drive/MyDrive/Project'

# 2) Load + preprocess + split
def vqa_rad_setup(path_project):
    json_file = "VQA_RAD Dataset Public.json"
    image_folder = "VQA_RAD Image Folder"
    json_path = os.path.join(path_project, json_file)
    image_dir  = os.path.join(path_project, image_folder)



    df = pd.read_json(json_path)
    # build full image paths
    df['image_path'] = df['image_name'].apply(lambda fn: os.path.join(image_dir, fn))

    total_rows     = len(df)
    print(f"Total rows in df_binary:      {total_rows}")


    num_organs = df['image_organ'].nunique()
    print(f"There are {num_organs} distinct organs.")

    # 3) if you want to see them all:
    print("Organ list:", df['image_organ'].unique())

    # filter to yes/no and map to 0/1
    df['answer'] = df['answer'].str.strip().str.lower()
    df_binary = df[df['answer'].isin(['yes', 'no'])].copy()
    df_binary['label'] = df_binary['answer'].map({'yes': 1, 'no': 0})

    # shuffle, select columns, print
    df_binary = (
        df_binary[['image_path', 'question', 'label']]
        .sample(frac=1, random_state=42)
        .reset_index(drop=True)
    )

    total_rows     = len(df_binary)
    print(f"Total rows in df_binary:      {total_rows}")
    return df_binary

In [ ]:
df_binary = vqa_rad_setup(path_project)

df_binary.head(10)

Total rows in df_binary:      2248
There are 3 distinct organs.
Organ list: ['HEAD' 'CHEST' 'ABD']
Total rows in df_binary:      1193


,image_path,question,label
0,/content/drive/MyDrive/Project/VQA_RAD Image F...,Are the patients' ribs symmetric on both sides?,0
1,/content/drive/MyDrive/Project/VQA_RAD Image F...,Are there cilia present at the level of alveoli?,0
2,/content/drive/MyDrive/Project/VQA_RAD Image F...,Is this coronal plane?,1
3,/content/drive/MyDrive/Project/VQA_RAD Image F...,Is the patient lying down?,1
4,/content/drive/MyDrive/Project/VQA_RAD Image F...,Do you see a cavitary lesion in this chest xray?,1
5,/content/drive/MyDrive/Project/VQA_RAD Image F...,Is there free air under the diaphragm?,0
6,/content/drive/MyDrive/Project/VQA_RAD Image F...,is there tracheal deviation?,0
7,/content/drive/MyDrive/Project/VQA_RAD Image F...,Is this in the lumbar vertebral level?,1
8,/content/drive/MyDrive/Project/VQA_RAD Image F...,Does this patient have a pneumothorax?,0
9,/content/drive/MyDrive/Project/VQA_RAD Image F...,Was this patient given IV contrast?,1


In [ ]:
!pip install transformers
!pip install open_clip_torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu1

In [ ]:
from PIL import Image
from tqdm import tqdm
import torch
import open_clip

# 1) Get preprocess from open_clip
import open_clip
model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms(
    'hf-hub:luhuitong/CLIP-ViT-L-14-448px-MedICaT-ROCO'
)

# 2) Image transformation function
def transform_image_pre(image_path):
    image = Image.open(image_path)
    if image.mode != 'RGB':
        image = image.convert('RGB')
    return preprocess_train(image)  # Already returns [3, 448, 448] tensor

# 3) Batch-process images
image_tensors = []
failed_images = []

for img_path in tqdm(df_binary['image_path'], desc="Processing images (448px)"):
    try:
        tensor = transform_image_pre(img_path)
        image_tensors.append(tensor)
    except Exception as e:
        print(f"❌ Skipped {img_path}: {e}")
        failed_images.append(img_path)

# 4) Stack tensors
if image_tensors:
    full_image_tensor = torch.stack(image_tensors, dim=0)
    print(f"\n✅ Processed {len(image_tensors)} images; skipped {len(failed_images)}.")
    print("📐 full_image_tensor.shape =", full_image_tensor.shape)
else:
    print("❌ No images were successfully processed.")


Processing images (448px): 100%|██████████| 1193/1193 [00:20<00:00, 58.78it/s]



✅ Processed 1193 images; skipped 0.
📐 full_image_tensor.shape = torch.Size([1193, 3, 448, 448])


In [ ]:
import torch
import numpy as np
from transformers import BertTokenizer


# Load tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
vocab_size = tokenizer.vocab_size
print(f"Loaded BERT tokenizer with vocab size: {vocab_size}")


# Set max sequence length for tokenization
max_seq_length = 64  # or any other value appropriate for your dataset

# Tokenize all questions
all_tokenized_texts = []
for description in df_binary['question']:
    tokenized_text = tokenizer(
        description,
        padding='max_length',
        truncation=True,
        max_length=max_seq_length,
        return_tensors="pt"
    )
    all_tokenized_texts.append(tokenized_text)

print(f"\nTokenized {len(all_tokenized_texts)} questions.")

# Stack all input_ids and attention_mask tensors
Text_tensor = torch.cat([item['input_ids'] for item in all_tokenized_texts], dim=0)  # [N, max_seq_length]
Text_attention_mask_tensor = torch.cat([item['attention_mask'] for item in all_tokenized_texts], dim=0)

# Final checks
print(f"\n✅ Shape of Text_tensor: {Text_tensor.shape}")                             # [N, max_seq_length]
print(f"✅ Shape of Attention Mask Tensor: {Text_attention_mask_tensor.shape}")       # [N, max_seq_length]
print(f"ℹ️  Max token ID: {Text_tensor.max().item()} < Vocab Size: {vocab_size}? {'Yes ✅' if Text_tensor.max().item() < vocab_size else 'No ❌'}")


Loaded BERT tokenizer with vocab size: 30522

Tokenized 1193 questions.

✅ Shape of Text_tensor: torch.Size([1193, 64])
✅ Shape of Attention Mask Tensor: torch.Size([1193, 64])
ℹ️  Max token ID: 29561 < Vocab Size: 30522? Yes ✅


In [ ]:
y_train_tensor = torch.tensor(df_binary['label'], dtype=torch.float32)

In [ ]:
first_path = df_binary['image_path'].iloc[0]

# 1) Open & print original size
img = Image.open(first_path)
print("Original PIL image size (w, h):", img.size)

# 2) Apply your preprocessing & print tensor shape
tensor = transform_image_pre(first_path)  # or preprocess_train(img)
print("After transform, tensor shape:", tensor.shape)


Original PIL image size (w, h): (1024, 1310)
After transform, tensor shape: torch.Size([3, 448, 448])


In [ ]:
import torch
from transformers import BertModel
import open_clip
from tqdm import tqdm

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms(
    'hf-hub:luhuitong/CLIP-ViT-L-14-448px-MedICaT-ROCO'
)
model = model.to(device)
model.eval()


bert_model = BertModel.from_pretrained("bert-base-uncased").to(device)
bert_model.eval()

Using device: cuda


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [ ]:


# Feature extraction function
def data_all_features(image_data, text_data, text_attention):
    image_features = []
    text_features = []

    for img, input_ids, attention_mask in tqdm(zip(image_data, text_data, text_attention), total=len(image_data)):
        img = img.unsqueeze(0).to(device)
        input_ids = input_ids.unsqueeze(0).to(device)
        attention_mask = attention_mask.unsqueeze(0).to(device)

        with torch.no_grad():
            # Get image features from CLIP
            #img_feat = model.get_image_features(pixel_values=img)  # [1, 1024]
            img_feat = model.encode_image(img)
            # Get text features from BERT
            txt_outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
            txt_feat = txt_outputs.pooler_output  # [1, 768]

        image_features.append(img_feat.squeeze(0))  # [1024]
        text_features.append(txt_feat.squeeze(0))   # [768]

    return torch.stack(image_features), torch.stack(text_features)

# Run feature extraction
image_features, text_features = data_all_features(
    full_image_tensor, Text_tensor, Text_attention_mask_tensor
)

# Shape check
print(f"✅ image_features shape: {image_features.shape}")  # [N, 1024]
print(f"✅ text_features shape: {text_features.shape}")    # [N, 768]

# Split
N = 75
Image_train = image_features[N:]
Image_test = image_features[:N]
Text_train = text_features[N:]
Text_test = text_features[:N]
y_train = y_train_tensor[N:].to(device)
y_test = y_train_tensor[:N].to(device)




100%|██████████| 1193/1193 [01:09<00:00, 17.08it/s]

✅ image_features shape: torch.Size([1193, 768])
✅ text_features shape: torch.Size([1193, 768])


In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Create dataloaders
batch_size = 128
train_dataset = TensorDataset(Image_train, Text_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(Image_test, Text_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


In [ ]:
print(train_loader.dataset.tensors[0].shape)
print(train_loader.dataset.tensors[1].shape)
print(train_loader.dataset.tensors[2].shape)

torch.Size([1118, 768])
torch.Size([1118, 768])
torch.Size([1118])


In [ ]:
import torch
import torch.nn as nn

class CoAttentionFusionClassifier(nn.Module):
    def __init__(
        self,
        image_dim: int,
        text_dim: int,
        fusion_dim: int = 512,
        num_heads: int = 8,
        dropout: float = 0.1,
    ):
        super().__init__()
        # project each modality into the same hidden space
        self.img_proj = nn.Linear(image_dim, fusion_dim)
        self.txt_proj = nn.Linear(text_dim, fusion_dim)

        # cross‑attention layers
        # image queries, text keys/values
        self.attn_img2txt = nn.MultiheadAttention(
            embed_dim=fusion_dim, num_heads=num_heads, dropout=dropout, batch_first=True
        )
        # text queries, image keys/values
        self.attn_txt2img = nn.MultiheadAttention(
            embed_dim=fusion_dim, num_heads=num_heads, dropout=dropout, batch_first=True
        )

        # classification head
        self.dropout    = nn.Dropout(dropout)
        self.classifier = nn.Linear(fusion_dim, 1)

    def forward(self, img_feat: torch.Tensor, txt_feat: torch.Tensor):
        """
        img_feat: (B, image_dim)
        txt_feat: (B, text_dim)
        """
        # 1) project and add sequence dim → (B, 1, fusion_dim)
        img = self.img_proj(img_feat).unsqueeze(1)
        txt = self.txt_proj(txt_feat).unsqueeze(1)

        # 2) co‑attention
        # img attends to text:
        img2txt, _ = self.attn_img2txt(query=img, key=txt, value=txt)
        # text attends to image:
        txt2img, _ = self.attn_txt2img(query=txt, key=img, value=img)

        # 3) fuse by averaging both attended outputs and pooling the sequence dim
        # each is (B,1,fusion_dim) → stack → (B,2,fusion_dim)
        fused_seq = torch.cat([img2txt, txt2img], dim=1)
        fused     = fused_seq.mean(dim=1)    # (B, fusion_dim)

        # 4) classification
        x      = self.dropout(fused)
        logit  = self.classifier(x).squeeze(1)  # (B,)
        return logit


In [ ]:

image_dim = image_features.shape[1]
text_dim  = text_features.shape[1]

In [ ]:
from sklearn.metrics import roc_auc_score
from itertools import product
lrs = [1e-3, 1e-4, 1e-5]
dropouts = [0.1, 0.2, 0.3]
n_epochs = 50 # fewer epochs for quick tuning
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2) helper to train & eval one config
def run_trial(lr, dropout, img2txt_mult, txt2img_mult):
    # re‑init model
    model = CoAttentionFusionClassifier(
        image_dim=image_dim,
        text_dim=text_dim,
        fusion_dim=512,
        num_heads=64,
        dropout=dropout
    ).to(device)

    # 2) Define Parameter Groups (this is where you change the LR multipliers):
    param_groups = [
        {'params': [p for n, p in model.named_parameters() if 'attn_img2txt' in n], 'lr': lr * img2txt_mult}, # Fixed multiplier
        {'params': [p for n, p in model.named_parameters() if 'attn_txt2img' in n], 'lr': lr * txt2img_mult}, # Fixed multiplier
        {'params': [p for n, p in model.named_parameters() if 'attn' not in n]}, # Default LR for other params
    ]

    # 3) Initialize Optimizer with Parameter Groups:
    opt = optim.Adam(param_groups, lr=lr)
    crit = nn.BCEWithLogitsLoss()
    best_f1 = 0.0
    best_acc = 0.0
    best_auroc = 0.0

    for epoch in range(1, n_epochs+1):
        # train one epoch
        model.train()
        for imgs, txts, labs in train_loader:
            imgs, txts = imgs.to(device), txts.to(device)
            labs = labs.to(device).float()
            opt.zero_grad()
            logits = model(imgs, txts)
            loss = crit(logits, labs)
            loss.backward()
            opt.step()

        # eval
        model.eval()
        all_p, all_l = [], []
        with torch.no_grad():
            for imgs, txts, labs in test_loader:
                imgs, txts = imgs.to(device), txts.to(device)
                logits = model(imgs, txts)
                preds = (torch.sigmoid(logits) > 0.5).long()
                all_p.extend(preds.cpu().tolist())
                all_l.extend(labs.tolist())

        f1 = f1_score(all_l, all_p)
        acc = accuracy_score(all_l, all_p)
        auroc = roc_auc_score(all_l, all_p)
        best_f1 = max(best_f1, f1)
        best_acc = max(best_acc, acc)
        best_auroc = max(best_auroc, auroc)

    return best_f1, best_acc, best_auroc

# 3) grid‐search
# You also need to define img2txt_mults and txt2img_mults
img2txt_mults = [0.5, 1.0, 2.0, 3.0]
txt2img_mults = [0.5, 1.0, 2.0, 5.0]

results = []
for lr, dp, img2txt_mult, txt2img_mult in product(lrs, dropouts, img2txt_mults, txt2img_mults):
    f1, acc, auroc = run_trial(lr, dp, img2txt_mult, txt2img_mult)  # Get all metrics
    print(f" → lr={lr:.0e}, dropout={dp:.1f}, img2txt_mult={img2txt_mult}, txt2img_mult={txt2img_mult} → best Val F1 = {f1:.4f}, best Val Acc = {acc:.4f}, best Val AUROC = {auroc:.4f}\n")
    results.append((f1, acc, auroc, lr, dp, img2txt_mult, txt2img_mult))  # Store all metrics

# 4) pick best
best_result = max(results, key=lambda x: x[0])  # Find best by F1 score
best_f1, best_acc, best_auroc, best_lr, best_dp, best_img2txt_mult, best_txt2img_mult = best_result

# Print Best Configuration with LR Multipliers and AUROC:
print(f">>> Best config by F1: lr={best_lr:.0e}, dropout={best_dp:.1f}, img2txt_mult={best_img2txt_mult}, txt2img_mult={best_txt2img_mult} with Val F1={best_f1:.4f} (Acc={best_acc:.4f}, AUROC={best_auroc:.4f})")

 → lr=1e-03, dropout=0.1, img2txt_mult=0.5, txt2img_mult=0.5 → best Val F1 = 0.6667, best Val Acc = 0.6933, best Val AUROC = 0.6891

 → lr=1e-03, dropout=0.1, img2txt_mult=0.5, txt2img_mult=1.0 → best Val F1 = 0.6667, best Val Acc = 0.6800, best Val AUROC = 0.6720

 → lr=1e-03, dropout=0.1, img2txt_mult=0.5, txt2img_mult=2.0 → best Val F1 = 0.6667, best Val Acc = 0.6800, best Val AUROC = 0.6774

 → lr=1e-03, dropout=0.1, img2txt_mult=0.5, txt2img_mult=5.0 → best Val F1 = 0.6977, best Val Acc = 0.6800, best Val AUROC = 0.6774

 → lr=1e-03, dropout=0.1, img2txt_mult=1.0, txt2img_mult=0.5 → best Val F1 = 0.6479, best Val Acc = 0.6667, best Val AUROC = 0.6656

 → lr=1e-03, dropout=0.1, img2txt_mult=1.0, txt2img_mult=1.0 → best Val F1 = 0.6667, best Val Acc = 0.6667, best Val AUROC = 0.6645

 → lr=1e-03, dropout=0.1, img2txt_mult=1.0, txt2img_mult=2.0 → best Val F1 = 0.6897, best Val Acc = 0.7067, best Val AUROC = 0.6998

 → lr=1e-03, dropout=0.1, img2txt_mult=1.0, txt2img_mult=5.0 → best V